In [ ]:
#r "nuget: XPlot.Plotly.Interactive, 4.0.6"
#r "nuget: FSharp.Data"

open FSharp.Data
open XPlot.Plotly

type Mandate = {
    name: string
    starting: int
    ending: int
}

let fhc1 = { name = "FHC"; starting = 1995; ending = 1999 }
let fhc2 = { name = "FHC"; starting = 1999; ending = 2003}
let lula1 = { name = "Lula"; starting = 2003; ending = 2007 }
let lula2 = { name = "Lula"; starting = 2007; ending = 2011 }
let dilma1 = { name = "Dilma"; starting = 2011; ending = 2015 }
let dilma2 = { name = "Dilma"; starting = 2015; ending = 2017 }
let temer = { name = "Temer"; starting = 2017; ending = 2018 }
let bolsonaro = { name = "Bolsonaro"; starting = 2018; ending = 2022 }

let mandates = [| fhc1; fhc2; lula1; lula2; dilma1; dilma2; temer; bolsonaro |]

let wb = WorldBankData.GetDataContext()

let br = wb.Countries.``Brazil``

let filter indicator starting ending  =
    seq {
            for year, value in indicator do
                yield year, value
        }
        |> Seq.filter(fun (year,_) -> year >= starting && year <= ending)
        |> Seq.toList
        |> List.unzip

let scatter indicator mandate =
    let years, values = filter indicator mandate.starting mandate.ending
    Scatter (name = mandate.name, x = years, y = values)

let chart title indicator =
    let plotter = scatter indicator
    mandates
    |> Seq.map plotter
    |> Chart.Plot
    |> Chart.WithTitle title

let candle indicator =
    let convert (_, values) =
        let opening = values |> Seq.head
        let closing = values |> Seq.last
        let high = values |> Seq.max
        let low = values |> Seq.min
        opening, high, low, closing
    
    mandates
    |> Seq.map ( fun mandate ->
        let opening, high, low, closing =
            filter indicator mandate.starting mandate.ending
            |> convert
        mandate.name, opening, high, low, closing
    )
    |> Chart.Candlestick


Installed Packages FSharp.Data, 4.2.8 XPlot.Plotly.Interactive, 4.0.6

In [ ]:
br.Indicators.``GDP growth (annual %)``
|> chart "Crescimento do PIB (% anual)" 

<!DOCTYPE html>

In [ ]:
br.Indicators.``GDP per capita (constant LCU)`` |> candle |> Chart.WithTitle "Variação do PIB per Capita"

<!DOCTYPE html>

In [ ]:
br.Indicators.``Inflation, consumer prices (annual %)``
|> chart "Inflação, preços ao consumidor (% anual)"

<!DOCTYPE html>

In [ ]:
br.Indicators.``Unemployment, total (% of total labor force) (modeled ILO estimate)``
|> chart "Desemprego"

<!DOCTYPE html>

In [ ]:
br.Indicators.``Access to electricity (% of population)``
|> chart "Acesso a Eletricidade"

<!DOCTYPE html>